In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer, LSTM, SimpleRNN, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop
import re
import sys

### Определяем архитектуру используемых нейросетей и заранее задаём размеры батчей

Для двухслойной LSTM был выбран больший размер батчей, т.к. обучение сети занимает большое время.

In [ ]:
rnn_batches = 32
gru_batches = 32
lstm1_batches = 32
lstm2_batches = 128

def rnn(X, y, shape):
  model = Sequential()
  model.add(InputLayer(input_shape = shape))
  model.add(Dropout(0.25))
  model.add(SimpleRNN(256))
  model.add(Dropout(0.2))
  model.add(Dense(y.shape[1], activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
  return model

def gru(X, y, shape):
  model = Sequential()
  model.add(InputLayer(input_shape = shape))
  model.add(Dropout(0.25))
  model.add(GRU(256, return_sequences = False, unroll = True))
  model.add(Dropout(0.2))
  model.add(Dense(y.shape[1], activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
  return model  

def lstm1(X,y, shape):
  model = Sequential()
  model.add(InputLayer(input_shape = shape))
  model.add(Dropout(0.25))
  model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2])))
  model.add(Dropout(0.2))
  model.add(Dense(y.shape[1], activation='softmax'))
  model.compile(loss ='categorical_crossentropy', optimizer ='adam')
  return model

def lstm2(X,y, shape):
  model = Sequential()
  model.add(InputLayer(input_shape = shape))
  model.add(Dropout(0.25))
  model.add(LSTM(256, return_sequences = True, input_shape = (X.shape[1], X.shape[2]))) #batch_input_shape = (lstm2_batches, X.shape[1], X.shape[2])
  model.add(Dropout(0.2))
  model.add(LSTM(256, return_sequences = False, stateful = False))
  model.add(Dropout(0.25))
  model.add(Dense(y.shape[1], activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
  return model

### Загружаем и обрабатываем датасет (текст)
При препроцессинге текст был приведён к общему регистру и из него были убраны: большинство знаков пунктуации, буквы латиницы (в исходном тексте изредка встречались короткие фразы на французском) и лишние пробелы.

Создаётся два словаря для сопоставления каждого встречающегося в тексте символа некоторому целому числу и наоборот. 
Входные данные кодируются по принципу one-hot encoding - единицы ставятся напротив встречающегося в данной позиции символа.

In [78]:
filename = "Dost_notepad.txt"
text1 = open(filename, 'r', encoding='utf-8').read()
text1 = text1.lower()
text1 = re.sub('[#()*:;018\-\'\"\nacdghimnopqrstuvxl]', '', text1)
text1 = re.sub('\s{2,}', ' ', text1)
chars = sorted(list(set(text1)))
char_to_int1 = dict((c, i) for i, c in enumerate(chars))
int_to_chars1 = dict((i, c) for i, c in enumerate(chars))

n_chars = len(text1)
n_vocab1 = len(chars)
print("Символьная длина отформатированного текста:", n_chars)
print("Всего встречается различных символов:", n_vocab1)
print("Эти символы:", chars)


fragments = []
next_chars = []
step = 1
length = 50

for k in range(0, len(text1) - length, step):
    fragments.append(text1[k : k + length])
    next_chars.append(text1[k + length])
print("Число фрагментов:", len(fragments))

X2 = np.zeros((len(fragments), length, n_vocab1), dtype=np.bool)
y2 = np.zeros((len(fragments), n_vocab1), dtype=np.bool) 
for m, fragment in enumerate(fragments):
    for t, char in enumerate(fragment):
        X2[m, t, char_to_int1[char]] = 1
    y2[m, char_to_int1[next_chars[m]]] = 1
print(X2.shape)
print(y2.shape)

Символьная длина отформатированного текста: 211342
Всего встречается различных символов: 38
Эти символы: [' ', '!', ',', '.', '?', 'e', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']
Число фрагментов: 211292
(211292, 50, 38)
(211292, 38)


### Вспомогательные функции

Функция sample используется при генерации текста вместо обычного np.argmax(), она обеспечивает некоторую случайность при генерации символов с помощью мультиномиального распределения и позволяет варьировать степень этой "разнообразности" с помощью temperature.

Функция generate отвечает непосредственно за генерацию текста, начиная с случайно выбранного отрывка исходного текста, и была вынесена сюда просто для удобства. За это удобство и нежелание повторять один и тот же код, однако, пришлось заплатить большим числом аргументов функции (чтобы не было конфликта между глобальными переменными двух разных датасетов).

In [ ]:
def sample(preds, temperature):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate(model, text, length, n_vocab, char_to_int, int_to_chars, temperature = 1.0):
  start = np.random.randint(0, len(text)- length - 1)
  sentence = text[start : start + length]
  print("Генерируем начиная с отрывка:")
  print("\"" + sentence + "\"") 
  print("Генерация: ")    
  for i in range(500):
    x_pred = np.zeros((1, length, n_vocab))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_to_int[char]] = 1.0
    prediction = model.predict(x_pred, verbose=0)
    #index = np.argmax(prediction)
    index = sample(prediction[0], temperature = temperature)
    result = int_to_chars[index]
    sys.stdout.write(result)
    sentence += result
    sentence = sentence[1:len(sentence)]
  print ("\n Генерация окончена.")

### Создаём и обучаем однослойную RNN сеть

In [ ]:
rnn_1layer = rnn(X2, y2, shape = (length, n_vocab1))

In [ ]:
epochs = 80

filepath="RNN-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

rnn_1layer.fit(X2, y2, epochs=epochs, batch_size=rnn_batches, callbacks=callbacks_list)

In [ ]:
filename = "RNN-weights-improvement-14-2.4017.hdf5"
rnn_1layer.load_weights(filename)
#opt = RMSprop(learning_rate=0.01)
rnn_1layer.compile(loss='categorical_crossentropy', optimizer='adam')

rnn_1layer.fit(X2, y2, epochs=epochs, batch_size=rnn_batches, callbacks=callbacks_list)

**Генерация однослойной RNN сети:**

In [93]:
filename = "RNN-weights-improvement-14-2.4017.hdf5"
rnn_1layer.load_weights(filename)
rnn_1layer.compile(loss='categorical_crossentropy', optimizer='adam')

generate(rnn_1layer, text1, length, n_vocab1, char_to_int1, int_to_chars1, temperature = 0.7)

Генерируем начиная с отрывка:
"баранам и проч., и проч. вот муравьи совершенно др"
Генерация: 
уго может быть по признаки на так на себя на и потому что на и выдь ставали  о воторин придомне положали не порому на и посторал я в потом на стали по подомал я на посте на по посторит любил за от не подомал я на поста и собельно и вы примей потом не посто не посто не старал я вы приз и на и потому что примо на и вы да от разимень не за от котором приво во все это продом ени на сто на она было по подем не посто не от разво призамил я но посел то на моне поставил я и на постовикал я продил на на 
 Генерация окончена.


### Создаём и обучаем однослойную LSTM сеть

In [ ]:
lstm_1layer = lstm1(X2, y2, shape = (length, n_vocab1))

In [ ]:
epochs = 100

filepath="LSTM1-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

lstm_1layer.fit(X2, y2, epochs=epochs, batch_size=lstm1_batches, callbacks=callbacks_list)

In [79]:
epochs = 1
filename = "LSTM1-weights-improvement-99-1.7102.hdf5"

filepath="LSTM1-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

lstm_1layer.fit(X2, y2, epochs=epochs, batch_size=lstm1_batches, callbacks=callbacks_list)

Epoch 1/1
211292/211292 [==============================] - 214s 1ms/step - loss: 1.7283

Epoch 00001: loss improved from inf to 1.72834, saving model to LSTM1-weights-improvement-01-1.7283.hdf5


**Генерация однослойной LSTM сети:**

In [104]:
filename = "LSTM1-weights-improvement-99-1.7102.hdf5"
lstm_1layer.load_weights(filename)
lstm_1layer.compile(loss='categorical_crossentropy', optimizer='adam')

generate(lstm_1layer, text1, length, n_vocab1, char_to_int1, int_to_chars1, temperature = 0.7)

Генерируем начиная с отрывка:
"ы. мало того с волнением и страстью будет говорить"
Генерация: 
 к подвали. неровнок ведь тот, значит, я воображал к селе ощущать, что у нас вам смеятия, потому что я тебе не мог тоже бывало что. да и не переплавилась, была вельне идетлы. своий вас, и потому вы только делать со плавить клавить себя, и в том, что я ее только много завязнув борогу за столо до попелокних каи видитиль уже не полюбил и сам с резортиченким к нечамими запротилась вас неньми, все будет потому что она уверите сордут, до тогда в содволе, потому что он не стал было положить ощещение на
 Генерация окончена.


### Создаём и обучаем однослойную GRU сеть

In [ ]:
gru_1layer = gru(X2, y2, shape = (length, n_vocab1))

In [ ]:
epochs = 30

filepath="GRU-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

gru_1layer.fit(X2, y2, epochs=epochs, batch_size=gru_batches, callbacks=callbacks_list)

**Генерация однослойной GRU сети:**

In [98]:
filename = "GRU-weights-improvement-30-1.8857.hdf5"
gru_1layer.load_weights(filename)
gru_1layer.compile(loss='categorical_crossentropy', optimizer='adam')

generate(gru_1layer, text1, length, n_vocab1, char_to_int1, int_to_chars1, temperature = 0.7)

Генерируем начиная с отрывка:
"ельно случалось страдать, но вы нисколько не уважа"
Генерация: 
ть, ала а вырвыл и на что она уме не свои желание может дойрет тебя на них соонаничей и меня, варительно потемится. ведь у не столы о вывела. а теловито отрочал, что он скавет только себе в дела оменно, не только припомнили меня за вам а рассказывался не смыхнуте совезунных хоторого, кокойно у достоил зачем выпертены и еще он был передо мной не за вс уви отвязаться. я в первой прилечны потому что о меня в комом. не было в векомиза. и и всего этого приервал сейка дурга те, когда ее у с безворгала
 Генерация окончена.


### Новый датасет: загрузка и обработка 

Три однослойных нейронных сети обучались на одном и том же тексте, для разнообразия используем для обучения двухслойной LSTM сети другой текст.

In [102]:
filename = "Idiot_notepad2.txt"
text2 = open(filename, 'r', encoding='utf-8').read()
text2 = text2.lower()
text2 = re.sub('[#()*:;0124678\-\'\"\nacdeghi\]\[\xa0mbfjlyznopqrstuvx]', '', text2)
text2 = re.sub('\s{2,}', ' ', text2)

chars = sorted(list(set(text2)))
char_to_int2 = dict((c, i) for i, c in enumerate(chars))
int_to_chars2 = dict((i, c) for i, c in enumerate(chars))

n_chars = len(text2)
n_vocab2 = len(chars)
print("Символьная длина отформатированного текста:", n_chars)
print("Всего встречается различных символов:", n_vocab2)
print("Эти символы:", chars)

length = 50
fragments = []
next_chars = []
step = 1

for k in range(0, len(text2) - length, step):
    fragments.append(text2[k : k + length])
    next_chars.append(text2[k + length])
print("Число фрагментов:", len(fragments))

X2 = np.zeros((len(fragments), length, n_vocab2), dtype=np.bool)
y2 = np.zeros((len(fragments), n_vocab2), dtype=np.bool)
for m, fragment in enumerate(fragments):
    for t, char in enumerate(fragment):
        X2[m, t, char_to_int2[char]] = 1
    y2[m, char_to_int2[next_chars[m]]] = 1
print(X2.shape)
print(y2.shape)

Символьная длина отформатированного текста: 1284941
Всего встречается различных символов: 38
Эти символы: [' ', '!', ',', '.', '?', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']
Число фрагментов: 1284891
(1284891, 50, 38)
(1284891, 38)


### Создаём и обучаем двухслойную LSTM сеть

In [107]:
lstm_2layers = lstm2(X2, y2, shape = ((length, n_vocab2)))

In [ ]:
epochs = 40

filepath="LSTM2-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

lstm_2layers.fit(X2, y2, epochs=epochs, batch_size=lstm2_batches, callbacks=callbacks_list)

In [ ]:
filename = "LSTM2-weights-improvement-04-1.4042.hdf5"
lstm_2layers.load_weights(filename)
epochs = 20
lstm_2layers.fit(X2, y2, epochs=epochs, batch_size=lstm2_batches, callbacks=callbacks_list)

**Генерация двухслойной LSTM сети:**

In [120]:
filename = "LSTM2-weights-improvement-19-1.7737.hdf5"
#optimizer = RMSprop(learning_rate=0.01)
lstm_2layers.load_weights(filename)
lstm_2layers.compile(loss='categorical_crossentropy', optimizer='adam')

generate(lstm_2layers, text2, length, n_vocab2, char_to_int2, int_to_chars2, temperature = 1.2)

Генерируем начиная с отрывка:
", пробормотал ганя, а кстати, скажите мне, как вы "
Генерация: 
я длма ш боже поссмер, то! объясняя клевку? впрочем, а именно... ксшелав аделаида? до рогожин жьл вварх, да и мечьов спотоинлягивал с, не одуже исет, хверял, хнибаг, много сезочек и мата и от просьой объявления ногой у радуката. она понятья участницу, нисколько бут свои низвы только отлюдите струю бохапили, и допросил однако же, в мешку! однако же, говорила, я это если уж любил истогич, мавичкой князь. с дя я с сила сплучше на тогодичусь, эту жизнь кнцзы? неншко аделаиды илы пряснешься, дин гося
 Генерация окончена.
